In [4]:
import pandas as pd

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

apipath = r'H:\\My Drive\\config\\hbqa.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)
secret_key = config['global']['OPENAI_API_KEY']
datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']

In [6]:
# This chunk was created to use with ChatGPT Front End to create prompt for generating QA.
# At this point of time there was no clarity around chunk size, number of question, types of question, \
# as per initial investigation cost of ChatGPT API was very high. So intially this Chunk was created. \
# Unfortuntely the main file is lost. But the prompt created from those chunks still exists. So, after\
#  reverse engineering, created Chunk from the Prompt text.

df_qa= pd.read_csv(datapath+'05-QAGen_ChatGPT_FE-Created-14Sep23-(ManualStep).csv') #This CSV is created from google drive file. It has Chunk info.

print(df_qa.shape)
df_qa = df_qa.loc[df_qa['QA'].notna(),:]
print(df_qa.shape)
df_qa.sample(4)

(256, 7)
(256, 7)


,Context_Len,Len,Section_Id,Chunk_Id,Prompt,Chunk,QA
237,2000,2551,Book01_179,699,"Write 5 question, corresponding answer, and th...","The Gandharva continued, 'Beholding his asylum...",Q1: Why did the Muni tie himself strongly with...
225,2000,1822,Book01_080,334,"Write 4 question, corresponding answer, and th...",Section LXXX\n(Sambhava Parva continued)\nVais...,Q1: Why did Kavya become angry and address Vri...
139,2000,2728,Book01_072,297,"Write 5 question, corresponding answer, and th...","Kanwa continued, 'And Sakra, thus addressed by...",Q1: Who commanded the god of the wind to be pr...
95,2000,3920,Book01_142,575,"Write 6 question, corresponding answer, and th...","Listen to me, O sinless king, as I answer thee...",Q1: What should kings be ready with and what s...


In [7]:
df_hbqa_manual = pd.DataFrame(columns=['Ques_Id','Chunk_Id','Section_Id', 'Question','Ref_Answer','Chunk','Reference'])
df_hbqa_manual

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference


In [8]:
rowid=0
for row in df_qa.index: # For dataset

    chunk = df_qa.loc[row,'Chunk'].strip()
    section_id = df_qa.loc[row,'Section_Id']
    chunk_id = df_qa.loc[row,'Chunk_Id']

    qa_text = df_qa.loc[row,'QA'].strip() #.replace('Questions:','')
    qas = qa_text.split('\n\n')
    # print((qas))
    for qa in qas: # For Each row, it has 5 question
        text1=qa.split('\n')
        # print(text1)
        if len(text1)==3:
            ques = text1[0][3:].strip()
            ans  = text1[1][3:].strip()
            ref  = text1[2][3:].strip()
        elif len(text1)==4:
            ques = text1[1][3:].strip()
            ans  = text1[2][3:].strip()
            ref  = text1[3][3:].strip()
        else:
            ques=""
            ans=""
        if ques!="":
            qa_row = rowid, chunk_id, section_id, ques, ans, chunk, ref
            #print(qa_row)
            
            df_hbqa_manual.loc[rowid] = qa_row
            rowid+=1
              


In [9]:
df_hbqa_manual

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference
0,0,193,Book01_046,Why did the Muni wander over the earth and wee...,The Muni wandered over the earth and wept loud...,"Sauti continued, 'The Muni, having said so unt...","""'O ye, being directed by my ancestors, I am r..."
1,1,193,Book01_046,Who did the Muni ask for a bride and why?,The Muni asked for a bride from the creatures ...,"Sauti continued, 'The Muni, having said so unt...","""My ancestors, afflicted with grief, have dire..."
2,2,193,Book01_046,Who informed Vasuki about the Muni's desire fo...,The snakes that had been set upon Jaratkaru's ...,"Sauti continued, 'The Muni, having said so unt...","""Then those snakes that had been set upon Jara..."
3,3,193,Book01_046,How did Vasuki respond when he heard about the...,"Vasuki took a maiden, who was decked with orna...","Sauti continued, 'The Muni, having said so unt...","""And the king of the snakes, hearing their wor..."
4,4,193,Book01_046,Why did the Rishi hesitate to accept the maide...,The Rishi hesitated to accept the maiden becau...,"Sauti continued, 'The Muni, having said so unt...","""And the Rishi, thinking her not to be of the ..."
...,...,...,...,...,...,...,...
1099,1099,15,Book01_002,Who did Duryodhana ask for assistance in the w...,"Duryodhana asked for troops, while Arjuna requ...",The extensive Parva that comes next is called ...,"""Blind to his own interests, the foolish Duryo..."
1100,1100,15,Book01_002,Who went as an envoy to the Pandavas from the ...,Sanjaya went as an envoy to the Pandavas from ...,The extensive Parva that comes next is called ...,"""Then is described how king Dhritarashtra of g..."
1101,1101,13,Book01_002,What is the name of the parva that describes t...,The name of the parva that describes the estab...,The second is the extensive parva called Sabha...,The name of Sabha Parva is given in this sente...
1102,1102,13,Book01_002,Who delivered the princes confined in the moun...,The princes confined in the mountain-pass by J...,The second is the extensive parva called Sabha...,The deliverance by Vasudeva is mentioned in th...


In [ ]:
# To avoid duplication with API generted questions. 
df_hbqa_manual['Ques_Id'] = df_hbqa_manual['Ques_Id']+10000
df_hbqa_manual['Chunk_Id'] = df_hbqa_manual['Chunk_Id']+10000

In [10]:
df_hbqa_manual['WordsInQues'] = df_hbqa_manual['Question'].str.split(' ').apply(len)
df_hbqa_manual['WordsInAns']  = df_hbqa_manual['Ref_Answer'].str.split(' ').apply(len)
df_hbqa_manual.head(5)

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns
0,0,193,Book01_046,Why did the Muni wander over the earth and wee...,The Muni wandered over the earth and wept loud...,"Sauti continued, 'The Muni, having said so unt...","""'O ye, being directed by my ancestors, I am r...",14,25
1,1,193,Book01_046,Who did the Muni ask for a bride and why?,The Muni asked for a bride from the creatures ...,"Sauti continued, 'The Muni, having said so unt...","""My ancestors, afflicted with grief, have dire...",10,26
2,2,193,Book01_046,Who informed Vasuki about the Muni's desire fo...,The snakes that had been set upon Jaratkaru's ...,"Sauti continued, 'The Muni, having said so unt...","""Then those snakes that had been set upon Jara...",10,20
3,3,193,Book01_046,How did Vasuki respond when he heard about the...,"Vasuki took a maiden, who was decked with orna...","Sauti continued, 'The Muni, having said so unt...","""And the king of the snakes, hearing their wor...",14,22
4,4,193,Book01_046,Why did the Rishi hesitate to accept the maide...,The Rishi hesitated to accept the maiden becau...,"Sauti continued, 'The Muni, having said so unt...","""And the Rishi, thinking her not to be of the ...",12,27


In [11]:
df_hbqa_manual.to_csv(datapath + '06-HBQA_Manual_with_Chunk.csv', index=None)

print(df_hbqa_manual.shape)
df_hbqa_manual.head(5)

(1104, 9)


,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns
0,0,193,Book01_046,Why did the Muni wander over the earth and wee...,The Muni wandered over the earth and wept loud...,"Sauti continued, 'The Muni, having said so unt...","""'O ye, being directed by my ancestors, I am r...",14,25
1,1,193,Book01_046,Who did the Muni ask for a bride and why?,The Muni asked for a bride from the creatures ...,"Sauti continued, 'The Muni, having said so unt...","""My ancestors, afflicted with grief, have dire...",10,26
2,2,193,Book01_046,Who informed Vasuki about the Muni's desire fo...,The snakes that had been set upon Jaratkaru's ...,"Sauti continued, 'The Muni, having said so unt...","""Then those snakes that had been set upon Jara...",10,20
3,3,193,Book01_046,How did Vasuki respond when he heard about the...,"Vasuki took a maiden, who was decked with orna...","Sauti continued, 'The Muni, having said so unt...","""And the king of the snakes, hearing their wor...",14,22
4,4,193,Book01_046,Why did the Rishi hesitate to accept the maide...,The Rishi hesitated to accept the maiden becau...,"Sauti continued, 'The Muni, having said so unt...","""And the Rishi, thinking her not to be of the ...",12,27
